In [1]:
!pip install tensorflow


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
os.chdir('/content/drive/My Drive/ML-class/project/dataset')


In [5]:
train_dir = "/content/drive/My Drive/ML-class/project/dataset/train"
test_dir = "/content/drive/My Drive/ML-class/project/dataset/test"

In [6]:
train_skin_types=os.listdir(train_dir)
train_skin_types


['Acne and Rosacea Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Atopic Dermatitis Photos',
 'Bullous Disease Photos',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Eczema Photos',
 'Exanthems and Drug Eruptions',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation',
 'Lupus and other Connective Tissue diseases',
 'Melanoma Skin Cancer Nevi and Moles',
 'Nail Fungus and other Nail Disease',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Seborrheic Keratoses and other Benign Tumors',
 'Systemic Disease',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
 'Urticaria Hives',
 'Vascular Tumors',
 'Vasculitis Photos',
 'Warts Molluscum and other Viral Infections']

In [7]:
number_of_skin_types = len(train_skin_types)

print("Number of skin types in train folder:", number_of_skin_types)


Number of skin types in train folder: 23


In [8]:
test_skin_types=os.listdir(test_dir)
test_skin_types


['Acne and Rosacea Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Atopic Dermatitis Photos',
 'Bullous Disease Photos',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Eczema Photos',
 'Exanthems and Drug Eruptions',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation',
 'Lupus and other Connective Tissue diseases',
 'Melanoma Skin Cancer Nevi and Moles',
 'Nail Fungus and other Nail Disease',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Seborrheic Keratoses and other Benign Tumors',
 'Systemic Disease',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
 'Urticaria Hives',
 'Vascular Tumors',
 'Vasculitis Photos',
 'Warts Molluscum and other Viral Infections']

In [9]:
number_of_skin_types = len(test_skin_types)

print("Number of skin types in test folder:", number_of_skin_types)


Number of skin types in test folder: 23


In [10]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split

# Define categories
categories = {
    'train': ['Acne and Rosacea Photos',
              'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
              'Atopic Dermatitis Photos',
              'Bullous Disease Photos',
              'Cellulitis Impetigo and other Bacterial Infections',
              'Eczema Photos',
              'Exanthems and Drug Eruptions',
              'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation'
              ],
    'test': ['Acne and Rosacea Photos',
              'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
             'Atopic Dermatitis Photos',
              'Bullous Disease Photos',
              'Cellulitis Impetigo and other Bacterial Infections',
             'Eczema Photos',
            'Exanthems and Drug Eruptions',
             'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation'
             ]
}

# Define paths
base_path = "/content/drive/My Drive/ML-class/project/dataset"
train_path = os.path.join(base_path, "train")
test_path = os.path.join(base_path, "test")

# Function to load data
def load_data(path, categories):
    data = {"image_path": [], "target": []}
    for idx, category in enumerate(categories):
        category_path = os.path.join(path, category)
        if os.path.isdir(category_path):  # Check if the path is a directory
            for img_name in os.listdir(category_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):  # Check for image files
                    img_path = os.path.join(category_path, img_name)
                    data["image_path"].append(img_path)
                    data["target"].append(idx)
    return pd.DataFrame(data)

# Load train and test data
train_df = load_data(train_path, categories['train'])
test_df = load_data(test_path, categories['test'])

# Function to preprocess and load images
def load_images(image_paths):
    images = []
    for img_path in image_paths:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (180, 180))
        img = img / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return images

# Take a sample of the data for quick testing
sample_train_df = train_df.sample(frac=0.3, random_state=42)
sample_test_df = test_df.sample(frac=0.1, random_state=42)

# Load images
train_images = load_images(sample_train_df['image_path'])
test_images = load_images(sample_test_df['image_path'])

# # Display example images
# example_imgs = random.sample(train_images, 6)
# fig, axes = plt.subplots(2, 3, figsize=(10, 8))
# for ax, img in zip(axes.ravel(), example_imgs):
#     ax.imshow(img)
#     ax.axis('off')
# plt.show()


In [11]:


# Convert lists of images to numpy arrays
train_images = np.array(train_images)
test_images = np.array(test_images)

# Convert lists of labels to numpy arrays
train_labels = np.array(sample_train_df['target'])
test_labels = np.array(sample_test_df['target'])

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Data Augmentation
data_augmentation = Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal'),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
])

# Create MobileNetV2 Model
base_model = tf.keras.applications.MobileNetV2(input_shape=(180, 180, 3), include_top=False, weights='imagenet')
base_model.trainable = False

number_of_classes = 10  # Update this based on your dataset
model = Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(number_of_classes, activation='softmax')
])

# # Compile the Model
model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])



9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
#Train the Model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), batch_size=32)



Epoch 1/20
46/46 [==============================] - 9s 86ms/step - loss: 2.3836 - accuracy: 0.1999 - val_loss: 2.0025 - val_accuracy: 0.3297
Epoch 2/20
46/46 [==============================] - 2s 46ms/step - loss: 1.9933 - accuracy: 0.3317 - val_loss: 1.9057 - val_accuracy: 0.3736
Epoch 3/20
46/46 [==============================] - 2s 51ms/step - loss: 1.8461 - accuracy: 0.3894 - val_loss: 1.8177 - val_accuracy: 0.3819
Epoch 4/20
46/46 [==============================] - 2s 53ms/step - loss: 1.7629 - accuracy: 0.4052 - val_loss: 1.7493 - val_accuracy: 0.4093
Epoch 5/20
46/46 [==============================] - 2s 52ms/step - loss: 1.6885 - accuracy: 0.4361 - val_loss: 1.7879 - val_accuracy: 0.3984
Epoch 6/20
46/46 [==============================] - 3s 57ms/step - loss: 1.6148 - accuracy: 0.4499 - val_loss: 1.7815 - val_accuracy: 0.4038
Epoch 7/20
46/46 [==============================] - 2s 53ms/step - loss: 1.5522 - accuracy: 0.4684 - val_loss: 1.7596 - val_accuracy: 0.4066
Epoch 8/20
46

In [ ]:
# Evaluate the Model
loss, accuracy = model.evaluate(test_images, test_labels)
print("Test accuracy: ", accuracy)

6/6 [==============================] - 0s 36ms/step - loss: 1.9060 - accuracy: 0.3765
Test accuracy:  0.37654322385787964


In [ ]:
def predict_skin_disease(image_path):
    # Define list of class names
    class_names = ['Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
                   'Acne and Rosacea Photos','Atopic Dermatitis Photos',
                    'Bullous Disease Photos',
                    'Cellulitis Impetigo and other Bacterial Infections',
             'Eczema Photos',
            'Exanthems and Drug Eruptions',
                   'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation'
                  ]

    # Load and preprocess image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (180, 180))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    # Make prediction on preprocessed image
    pred = model.predict(img)
    predicted_class_index = np.argmax(pred)
    predicted_class_name = class_names[predicted_class_index]

    return predicted_class_name, pred

img_path = test_dir+'/Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions/actinic-cheilitis-sq-cell-lip-6.jpg'
predicted_class, pred = predict_skin_disease(img_path)
print(predicted_class)
print("Prediction probabilities:", pred)



1/1 [==============================] - 1s 766ms/step
Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions
Prediction probabilities: [[0.41593158 0.21409215 0.0017711  0.13992111 0.02151102 0.12241213
  0.00338511 0.00317926 0.00095172 0.07684486]]


In [ ]:
img_path2 = test_dir+'/Acne and Rosacea Photos/07PerioralDermEye.jpg'
predicted_class, pred = predict_skin_disease(img_path2)
print(predicted_class)
print("Prediction probabilities:", pred)


1/1 [==============================] - 0s 23ms/step
Acne and Rosacea Photos
Prediction probabilities: [[0.08657448 0.42035523 0.01674657 0.06881848 0.03628497 0.2367955
  0.03894079 0.04183753 0.01202457 0.04162195]]


In [ ]:
!pip install scikit-image


**SVM**

In [12]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


In [18]:
# Defining the parameters grid for GridSearchCV
param_grid={'C':[0.1,1,10,100],
			'gamma':[0.0001,0.001,0.1,1],
			'kernel':['rbf','poly']}

# Creating a support vector classifier
svc=svm.SVC(probability=True)

# Creating a model using GridSearchCV with the parameters grid
model=GridSearchCV(svc,param_grid)


In [20]:
# Define the SVM classifier
# svc = SVC()

# Train the SVM classifier
svc.fit(X_train.reshape(-1, 180 * 180 * 3), y_train)

# Evaluate the SVM classifier on the validation set
y_pred = svc.predict(X_val.reshape(-1, 180 * 180 * 3))
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy on validation set:", accuracy)

# Evaluate the SVM classifier on the test set
y_pred = svc.predict(test_images.reshape(-1, 180 * 180 * 3))
accuracy = accuracy_score(y_pred, test_labels)
print("Accuracy on test set:", accuracy)

# Print the classification report
print(classification_report(test_labels, y_pred, target_names=['Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
                   'Acne and Rosacea Photos','Atopic Dermatitis Photos',
                    'Bullous Disease Photos',
                    'Cellulitis Impetigo and other Bacterial Infections', 'Eczema Photos',
              'Exanthems and Drug Eruptions',
               'Hair Loss Photos Alopecia and other Hair Diseases',
 'Herpes HPV and other STDs Photos',
 'Light Diseases and Disorders of Pigmentation'
              ]))


Accuracy on validation set: 0.3708791208791209
Accuracy on test set: 0.36419753086419754
                                                                    precision    recall  f1-score   support

Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions       0.49      0.47      0.48        38
                                           Acne and Rosacea Photos       0.30      0.68      0.42        31
                                          Atopic Dermatitis Photos       0.00      0.00      0.00        12
                                            Bullous Disease Photos       0.00      0.00      0.00         9
                Cellulitis Impetigo and other Bacterial Infections       0.00      0.00      0.00         4
                                                     Eczema Photos       0.37      0.74      0.49        27
                                      Exanthems and Drug Eruptions       0.00      0.00      0.00        17
                 Hair Loss Photos Alopecia and

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# Define the test image path
path= test_dir+'/Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions/actinic-cheilitis-sq-cell-lip-6.jpg'

# Load the test image
test_image = cv2.imread(path)

# Preprocess the test image
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
test_image = cv2.resize(test_image, (180, 180))
test_image = test_image / 255.0  # Normalize pixel values to [0, 1]

# Reshape the test image to a 2D array
test_image = test_image.reshape(-1, 180 * 180 * 3)

# Predict the category of the test image
predicted_category = svc.predict(test_image)[0]

print(predicted_category)

# Print the predicted category
print("Predicted category:", categories['test'][predicted_category])


1
Predicted category: Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions


**Reference:**

1. https://www.kaggle.com/code/ishaanchandak301/skin-detection-acc-98
2. https://www.geeksforgeeks.org/image-classification-using-support-vector-machine-svm-in-python/
